In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.4'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.4.0-dev20201007


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

start = time.time()
model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)

Epoch 1/10


/home/spai/.conda/envs/tf_nightly/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_2:1", shape=(None,), dtype=int32), values=Tensor("PartitionedCall_2:0", shape=(None, 300), dtype=float32), dense_shape=Tensor("PartitionedCall_2:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/home/spai/.conda/envs/tf_nightly/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall_2:4", shape=(None,), dtype=int32), values=Tensor("PartitionedCall_2:3", shape=(None, 300), dtype=float32), dense_shape=Tensor("PartitionedCall_2:5", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may 

     28/Unknown - 2s 77ms/step - loss: 13790.8828
Epoch 00001: loss improved from inf to 13395.02441, saving model to ./chkpt1
29/29 [==============================] - 2s 77ms/step - loss: 13395.0244
Epoch 2/10
28/29 [===========================>..] - ETA: 0s - loss: 12973.6162
Epoch 00002: loss improved from 13395.02441 to 12788.68945, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12788.6895
Epoch 3/10
28/29 [===========================>..] - ETA: 0s - loss: 12630.3213
Epoch 00003: loss improved from 12788.68945 to 12510.76367, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12510.7637
Epoch 4/10
28/29 [===========================>..] - ETA: 0s - loss: 12432.0020
Epoch 00004: loss improved from 12510.76367 to 12344.02539, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12344.0254
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 12299.4893
9 triples 

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 55s 269ms/step
Time taken: 55.378262758255005


(329.49283197964576,
 0.20089428661245567,
 0.12388687738526274,
 0.3540708484196105)

## Load Checkpoint and evaluate

In [5]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     scoring_type='TransE')
loaded_model.load_weights('./chkpt1')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 55s 266ms/step
Time taken: 54.866934061050415


(329.49283197964576,
 0.20089428661245567,
 0.12388687738526274,
 0.3540708484196105)

# Training/eval with partition 

## Training with RandomEdges partitioner

In [3]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)

# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)

_split: memory before: 896.0Bytes, after: 12.974MB, consumed: 12.974MB; exec time: 31.264s


In [4]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)

In [5]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

Epoch 1/10
274/274 [==============================] - 17s 60ms/step - loss: 1583.6094
Epoch 2/10
274/274 [==============================] - 15s 56ms/step - loss: 1408.4923
Epoch 3/10
274/274 [==============================] - 16s 57ms/step - loss: 1257.2207
Epoch 4/10
274/274 [==============================] - 15s 56ms/step - loss: 1132.7302
Epoch 5/10
274/274 [==============================] - 15s 55ms/step - loss: 1031.3352
Epoch 6/10
274/274 [==============================] - 15s 56ms/step - loss: 948.8479
Epoch 7/10
274/274 [==============================] - 15s 55ms/step - loss: 880.7571
Epoch 8/10
274/274 [==============================] - 15s 56ms/step - loss: 823.7659
Epoch 9/10
274/274 [==============================] - 15s 56ms/step - loss: 775.5018
Epoch 10/10
274/274 [==============================] - 15s 57ms/step - loss: 734.1245
167.24223685264587


In [7]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 129s 2s/step
Time taken: 129.43282413482666


(526.5985663959292, 0.09950692531469157, 0.0, 0.2718710245620902)

## Save and Load

In [8]:
partitioned_model.save_weights('./best_model')

In [9]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [10]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=loaded_part_model.data_indexer)

start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 133s 3s/step
Time taken: 133.1391897201538


(526.5985663959292, 0.09950692531469157, 0.0, 0.2718710245620902)

# Training/eval with partition (default Partitioning Approach)

In [3]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [4]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

_split: memory before: 896.0Bytes, after: 12.862MB, consumed: 12.861MB; exec time: 82.599s
Epoch 1/10
277/277 [==============================] - 11s 38ms/step - loss: 1704.0818
Epoch 2/10
277/277 [==============================] - 9s 33ms/step - loss: 1609.8291
Epoch 3/10
277/277 [==============================] - 9s 34ms/step - loss: 1520.3560
Epoch 4/10
277/277 [==============================] - 9s 33ms/step - loss: 1439.7467
Epoch 5/10
277/277 [==============================] - 9s 33ms/step - loss: 1368.3000
Epoch 6/10
277/277 [==============================] - 9s 34ms/step - loss: 1305.2173
Epoch 7/10
277/277 [==============================] - 9s 33ms/step - loss: 1249.3754
Epoch 8/10
277/277 [==============================] - 9s 33ms/step - loss: 1199.8024
Epoch 9/10
277/277 [==============================] - 9s 34ms/step - loss: 1155.5101
Epoch 10/10
277/277 [==============================] - 9s 33ms/step - loss: 1115.8169
184.2071602344513


In [5]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 130s 2s/step
Time taken: 130.32165026664734


(813.9855661023583, 0.08932181231265608, 0.0, 0.24018984245033762)

## Filtered evaluation

In [6]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
53/53 [==============================] - 289s 5s/step
Time taken: 289.215669631958


(689.3758684802818,
 0.19849747101114806,
 0.13976416479107545,
 0.3149525393874156)